<a href="https://colab.research.google.com/github/uless/Fake-News-Diffusion/blob/master/waves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
!pip install unidecode
!pip install py2neo

In [2]:
import os
import pandas as pd
from pathlib import Path
from py2neo import Graph
import time
import unidecode
import difflib
from functools import reduce
from google.colab import files

In [3]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [4]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

In [5]:
df = pd.read_excel("COVID_zipcodes.xlsx", index_col=0)

In [6]:
df.head()

,90pctplus_Counties,75pctplus_Counties,S1_ZipCode,S2_County_self_report,COVID_waves1to3_zipcodefocused__S1_ZipCode,state,county,resRatio,placeName,code,population
PROLIFIC_PID,,,,,,,,,,,
55a55bd7fdf99b790558627c,Hampden County,Hampden County,01001,hampden,1001,Massachusetts,Hampden County,1.0,Agawam,MA,469116.0
55f9df5f4f5140000e296836,Hampden County,Hampden County,01040,United States,1040,Massachusetts,Hampden County,1.0,Holyoke,MA,469116.0
5c7489824f23e50015134f02,Hampshire County,Hampshire County,1060,US,1060,Massachusetts,Hampshire County,1.0,Northampton,MA,161159.0
5dc091fa9be08f03fb01a816,Hampden County,Hampden County,1089,Hampden,1089,Massachusetts,Hampden County,1.0,West Springfield,MA,469116.0
5e2298499138e055cfd97ae0,Hampden County,Hampden County,1108,HAMPDEN,1108,Massachusetts,Hampden County,1.0,Springfield,MA,469116.0


In [7]:
df = df.astype({"S1_ZipCode": str})
df['S1_ZipCode'] = df['S1_ZipCode'].apply(lambda x: x if len(x) != 4 else '0' + x)

In [9]:
df.head()

,90pctplus_Counties,75pctplus_Counties,S1_ZipCode,S2_County_self_report,COVID_waves1to3_zipcodefocused__S1_ZipCode,state,county,resRatio,placeName,code,population
PROLIFIC_PID,,,,,,,,,,,
55a55bd7fdf99b790558627c,Hampden County,Hampden County,01001,hampden,1001,Massachusetts,Hampden County,1.0,Agawam,MA,469116.0
55f9df5f4f5140000e296836,Hampden County,Hampden County,01040,United States,1040,Massachusetts,Hampden County,1.0,Holyoke,MA,469116.0
5c7489824f23e50015134f02,Hampshire County,Hampshire County,01060,US,1060,Massachusetts,Hampshire County,1.0,Northampton,MA,161159.0
5dc091fa9be08f03fb01a816,Hampden County,Hampden County,01089,Hampden,1089,Massachusetts,Hampden County,1.0,West Springfield,MA,469116.0
5e2298499138e055cfd97ae0,Hampden County,Hampden County,01108,HAMPDEN,1108,Massachusetts,Hampden County,1.0,Springfield,MA,469116.0


In [12]:
df_zip = df.query("S1_ZipCode != ' '").copy()
df_zip['S1_ZipCode'] = df_zip['S1_ZipCode'].astype(int)

In [13]:
zip_unique = df_zip['S1_ZipCode'].unique()
len(zip_unique)

1752

In [14]:
df.drop_duplicates(inplace=True)

# Wave 1 Data

In [21]:
query = """
MATCH (p:PostalCode{name:$zip_code})-[i:IN]->(a2:Admin2)-[:IN]->(a1:Admin1),
      (a2)<-[:REPORTED_IN]-(c:Cases{source:'JHU'}),
      (a2)-[:HAS_DEMOGRAPHICS]->(d:Demographics{aggregationLevel: 'Admin2'}) 
      
WHERE c.date >= date($startDate) AND c.date <= date($endDate)
      
RETURN  a1.name AS state, a2.name AS county, i.resRatio AS resRatio, p.placeName AS placeName, a1.code AS code,
       $zip_code AS S1_ZipCode,
       d.totalPopulation as population, 
       c.date AS date,
       c.cases AS cases, c.deaths AS deaths,
       c.cases*100000.0/d.totalPopulation AS caseRate, 
       c.deaths*100000.0/d.totalPopulation AS deathRate
"""

In [22]:
data_W1= pd.concat((graph.run(query, zip_code=row.S1_ZipCode, 
                                              startDate='2020-01-31', endDate='2020-03-09').to_data_frame() 
                                   for row in df.itertuples()))

In [23]:
data_W2= pd.concat((graph.run(query, zip_code=row.S1_ZipCode, 
                                              startDate='2020-03-30', endDate='2020-05-06').to_data_frame() 
                                   for row in df.itertuples()))

In [24]:
data_W3= pd.concat((graph.run(query, zip_code=row.S1_ZipCode, 
                                              startDate='2020-07-09', endDate='2020-08-13').to_data_frame() 
                                   for row in df.itertuples()))

In [25]:
data_W4= pd.concat((graph.run(query, zip_code=row.S1_ZipCode, 
                                              startDate='2020-09-14', endDate='2020-10-21').to_data_frame() 
                                   for row in df.itertuples()))

In [26]:
data_W5= pd.concat((graph.run(query, zip_code=row.S1_ZipCode, 
                                              startDate='2020-12-25', endDate='2021-02-11').to_data_frame() 
                                   for row in df.itertuples()))

In [27]:
data_W6= pd.concat((graph.run(query, zip_code=row.S1_ZipCode, 
                                              startDate='2021-02-25', endDate='2020-04-13').to_data_frame() 
                                   for row in df.itertuples()))

In [28]:
data_W1.to_csv("COVID_wave1_zipcodefocused.csv", index=False)
data_W2.to_csv("COVID_wave2_zipcodefocused.csv", index=False)
data_W6.to_csv("COVID_wave6_zipcodefocused.csv", index=False)

In [32]:
data_W3.to_csv("COVID_wave3_zipcodefocused.csv", index=False)
data_W4.to_csv("COVID_wave4_zipcodefocused.csv", index=False)
data_W5.to_csv("COVID_wave5_zipcodefocused.csv", index=False)

In [ ]:
files.download("COVID_wave2_zipcodefocused.csv") 

In [29]:
data_W2.to_csv("COVID_wave2_zipcodefocused.csv", index=False)

In [ ]:
data_W1 = data_W1.sort_values(by=["S1_ZipCode", "county","date"])
data_W1['cases_diff'] = data_W1.groupby(['S1_ZipCode', 'country'])['cases'].diff().fillna(0)
data_W1['deaths_diff'] = data_W1.groupby(['S1_ZipCode', 'country'])['deaths'].diff().fillna(0)
data_W1['caseRate_diff'] = data_W1.groupby(['S1_ZipCode', 'country'])['caseRate'].diff().fillna(0)
data_W1['deathRate_diff'] = data_W1.groupby(['S1_ZipCode', 'country'])['deathRate'].diff().fillna(0)